# Installing required packages

In [ ]:
!pip install --upgrade accelerate
!pip install datasets
!pip install transformers==4.27.0
!pip install evaluate seqeval

In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, Features, Value, ClassLabel, Sequence, load_dataset
import evaluate
from seqeval.metrics import classification_report
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline
import warnings

from google.colab import drive
drive.mount('/content/drive')

# Processing source file

In [ ]:
enm = pd.read_excel('/content/enm1930_ner.xlsx')
enm['tokens'] = enm['tokens'].apply(lambda x: x.replace("'", ''))
enm['tokens'] = enm['tokens'].apply(lambda x: x.replace("[", ''))
enm['tokens'] = enm['tokens'].apply(lambda x: x.replace("]", ''))
enm['tokens'] = enm['tokens'].apply(lambda x: x.replace(",", ''))
enm['tags'] = enm['tags'].apply(lambda x: x.replace("]", ''))
enm['tags'] = enm['tags'].apply(lambda x: x.replace("[", ''))
enm['tags'] = enm['tags'].apply(lambda x: x.replace("'", ''))
enm['tags'] = enm['tags'].apply(lambda x: x.replace(",", ''))
enm['split_split_sent'] = enm['tokens'].apply(lambda x: x.split())
enm['split_ner'] = enm['tags'].apply(lambda x: x.split())

In [ ]:
def to_json(x, y):
  global enm_json
  enm_json.append({'sentence': x, 'tags': y})

enm_json = []
enm.apply(lambda x: to_json(x['split_split_sent'], x['split_ner']), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
463    None
464    None
465    None
466    None
467    None
Length: 468, dtype: object

In [ ]:
ds_IOB2 = pd.DataFrame(enm_json, columns = ['tokens', 'ner_tags_labels'])

In [ ]:
tag_ner = []
for d in enm_json:
  for t in d['tags']:
    if t not in tag_ner:
      tag_ner.append(t)

In [ ]:
tags_clean_dial = []

for d in enm_json:
  new_tags = []
  for i in range(len(d['tags'])):
    if d['tags'][i].startswith('B-'):
      new_tags.append('B-DIAL')
    elif d['tags'][i] == 'O':
      new_tags.append('O')
    else:
      new_tags.append('I-DIAL')
    
  tags_clean_dial.append({'sentence': d['sentence'], 'tags': new_tags})
    
ds_IOB2_dial = pd.DataFrame(tags_clean_dial, columns = ['sentence', 'tags'])

# Define the model

In [ ]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Defining a function that transforms the input

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["sentence"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids: 
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Prepare function for evaluating

In [ ]:
seqeval = evaluate.load("seqeval")

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
warnings.filterwarnings('ignore')

# Create dataset from our data

In [ ]:
ner_tags_labels_ = ['O', 'B-DIAL', 'I-DIAL']
ds_features = Features({'sentence':  Sequence(Value("string")),
                        'tags': Sequence(ClassLabel(names=ner_tags_labels_))})

dataset = Dataset.from_pandas(ds_IOB2_dial, features=ds_features)
dataset_splitted = dataset.train_test_split(test_size=0.5, seed=22)

In [ ]:
ds_tokenized = dataset_splitted.map(tokenize_and_align_labels, batched=True)
label_list = ds_tokenized["train"].features["tags"].feature.names
id2label = {i: label_list[i] for i in range(len(label_list))}
label2id = {label_list[i]: i for i in range(len(label_list))}

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

# Load the model which was trained on Zapadnodvisk data

In [ ]:
!unzip /content/drive/MyDrive/model_roberta_one.zip -d model_roberta

Archive:  /content/drive/MyDrive/model_roberta_one.zip
   creating: model_roberta/content/xlm_roberta_base_dial_V2/checkpoint-2000/
  inflating: model_roberta/content/xlm_roberta_base_dial_V2/checkpoint-2000/tokenizer.json  
  inflating: model_roberta/content/xlm_roberta_base_dial_V2/checkpoint-2000/rng_state.pth  
  inflating: model_roberta/content/xlm_roberta_base_dial_V2/checkpoint-2000/scheduler.pt  
  inflating: model_roberta/content/xlm_roberta_base_dial_V2/checkpoint-2000/optimizer.pt  
  inflating: model_roberta/content/xlm_roberta_base_dial_V2/checkpoint-2000/special_tokens_map.json  
  inflating: model_roberta/content/xlm_roberta_base_dial_V2/checkpoint-2000/tokenizer_config.json  
  inflating: model_roberta/content/xlm_roberta_base_dial_V2/checkpoint-2000/trainer_state.json  
  inflating: model_roberta/content/xlm_roberta_base_dial_V2/checkpoint-2000/config.json  
  inflating: model_roberta/content/xlm_roberta_base_dial_V2/checkpoint-2000/training_args.bin  
  inflating: mod

In [ ]:
model = AutoModelForTokenClassification.from_pretrained('/content/model_roberta/content/xlm_roberta_base_dial_V2/checkpoint-2000', num_labels=len(label_list), id2label=id2label, label2id=label2id)

# Evaluate model which wasn't yet trained on Opocka's data

In [ ]:
trainer = Trainer(model, tokenizer=tokenizer, data_collator=data_collator)
ds_tokenized_full = dataset.map(tokenize_and_align_labels, batched=True)
output = trainer.predict(ds_tokenized_full.remove_columns(['sentence', 'tags']))
predictions, labels, metrics = output
predictions = np.argmax(predictions, axis=2)

true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
print(classification_report(true_labels, true_predictions))

Map:   0%|          | 0/468 [00:00<?, ? examples/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


              precision    recall  f1-score   support

        DIAL       0.55      0.11      0.18       520

   micro avg       0.55      0.11      0.18       520
   macro avg       0.55      0.11      0.18       520
weighted avg       0.55      0.11      0.18       520



# Train

In [ ]:
training_args = TrainingArguments(
    output_dir="xlm_roberta_base_dial_V2_opochka",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=25,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tokenized["train"],
    eval_dataset=ds_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,No log,0.399763,0.545113,0.594262,0.568627,0.836018
200,No log,0.424009,0.610236,0.635246,0.622490,0.860176
300,No log,0.490914,0.613546,0.631148,0.622222,0.860176
400,No log,0.537483,0.634855,0.627049,0.630928,0.866764
500,0.232900,0.641099,0.608527,0.643443,0.625498,0.860176
600,0.232900,0.648002,0.622951,0.622951,0.622951,0.863104
700,0.232900,0.716852,0.587361,0.647541,0.615984,0.853587


TrainOutput(global_step=750, training_loss=0.18070709482828776, metrics={'train_runtime': 342.2928, 'train_samples_per_second': 17.091, 'train_steps_per_second': 2.191, 'total_flos': 60180453368640.0, 'train_loss': 0.18070709482828776, 'epoch': 25.0})

# Evaluate

In [ ]:
best_model_from_training_testing = '/content/xlm_roberta_base_dial_V2_opochka/checkpoint-400'
best_model= AutoModelForTokenClassification.from_pretrained(best_model_from_training_testing, num_labels=len(label_list), id2label=id2label, label2id=label2id)
trainer = Trainer(best_model, tokenizer=tokenizer, data_collator=data_collator)
output = trainer.predict(ds_tokenized['test'].remove_columns(['sentence', 'tags']))
predictions, labels, metrics = output
predictions = np.argmax(predictions, axis=2)

true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
print(classification_report(true_labels, true_predictions))

              precision    recall  f1-score   support

        DIAL       0.63      0.63      0.63       244

   micro avg       0.63      0.63      0.63       244
   macro avg       0.63      0.63      0.63       244
weighted avg       0.63      0.63      0.63       244



# Save model to disk

In [ ]:
!zip -r /content/drive/MyDrive/model_roberta_one_opochka.zip /content/xlm_roberta_base_dial_V2_opochka/checkpoint-400

  adding: content/xlm_roberta_base_dial_V2_opochka/checkpoint-400/ (stored 0%)
  adding: content/xlm_roberta_base_dial_V2_opochka/checkpoint-400/config.json (deflated 50%)
  adding: content/xlm_roberta_base_dial_V2_opochka/checkpoint-400/rng_state.pth (deflated 28%)
  adding: content/xlm_roberta_base_dial_V2_opochka/checkpoint-400/training_args.bin (deflated 48%)
  adding: content/xlm_roberta_base_dial_V2_opochka/checkpoint-400/trainer_state.json (deflated 67%)
  adding: content/xlm_roberta_base_dial_V2_opochka/checkpoint-400/tokenizer_config.json (deflated 49%)
  adding: content/xlm_roberta_base_dial_V2_opochka/checkpoint-400/scheduler.pt (deflated 48%)
  adding: content/xlm_roberta_base_dial_V2_opochka/checkpoint-400/tokenizer.json (deflated 76%)
  adding: content/xlm_roberta_base_dial_V2_opochka/checkpoint-400/pytorch_model.bin (deflated 28%)
  adding: content/xlm_roberta_base_dial_V2_opochka/checkpoint-400/special_tokens_map.json (deflated 52%)
  adding: content/xlm_roberta_base_di